In [16]:
%matplotlib qt

# std
import os
from os import path
import time
import pickle as pkl
from importlib import reload  

# 3p
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# prj
os.chdir("..")
from src import helpers, configuration, filtering, bandpower, plots, statistics as stats
from src.configuration import cfg, cfg_mr
os.chdir('analysis-adelie')

# mne
import mne

from scipy.io import loadmat  # this is the SciPy module that loads mat-files
from datetime import datetime, date, time

# statistics
from scipy.stats import ttest_ind

# plots
import matplotlib

font = {'family':'DejaVu Sans', 'weight':'normal', 'size':10}
matplotlib.rc('font', **font)

#### Load data

In [9]:
from importlib import reload  
reload(bandpower)
reload(configuration)
cfg_mr

{'paths': {'base': 'C:/Users/adeli/OneDrive/Documents/GitHub/data/AlphaTheta',
  'subjects': {'mr': {'prefix': '/MatthieuRicard',
    'recordings': {'baseline': ['baseline1', 'baseline2'],
     'meditation': ['meditation']}}}},
 'columns_to_remove': ['nois', 'sync', 'STI 014'],
 'default_signal_crop': slice(3000, -3000, None),
 'sampling_frequency': 250,
 'bands': {'gamma': [40, 100],
  'beta': [12, 40],
  'alpha': [8, 12],
  'theta': [4, 8],
  'delta': [1, 4]}}

In [17]:
paths = ["../../data/AlphaTheta/MatthieuRicard/MR_20150512_RestingState/MR_12052015.raw",
         "../../data/AlphaTheta/MatthieuRicard/MR_20150511_Meditation/Pre_injection_rest/MR_20150511_RestingState.raw", 
         "../../data/AlphaTheta/MatthieuRicard/MR_20150511_Meditation/Post_injection_meditation/MR_20150511_Meditation.raw"]

In [11]:
data_baseline1 = mne.io.read_raw_egi(paths[0], preload=False)

Reading EGI header from ../../data/AlphaTheta/MatthieuRicard/MR_20150512_RestingState/MR_12052015.raw...
    Reading events ...
    Assembling measurement info ...
    Synthesizing trigger channel "STI 014" ...
    Excluding events {nois, sync} ...


In [18]:
data_baseline2 = mne.io.read_raw_egi(paths[1], preload=False)

Reading EGI header from ../../data/AlphaTheta/MatthieuRicard/MR_20150511_Meditation/Pre_injection_rest/MR_20150511_RestingState.raw...
    Reading events ...
    Assembling measurement info ...
    Synthesizing trigger channel "STI 014" ...
    Excluding events {sync} ...


<ipython-input-18-9b3427931c12>:1: RuntimeWarning: Did not find any event code with more than one event.
  data_baseline2 = mne.io.read_raw_egi(paths[1], preload=False)


In [ ]:
data_meditation  = mne.io.read_raw_egi(paths[2], preload=False)

#### Preprocess

In [12]:
data_baseline1_df = data_baseline1.to_data_frame(index=['time'], time_format='timedelta')

In [19]:
data_baseline2_df = data_baseline2.to_data_frame(index=['time'], time_format='timedelta')

MemoryError: Unable to allocate 1.87 GiB for an array with shape (967758, 259) and data type float64

In [ ]:
data_meditation_df = data_meditation.to_data_frame(index=['time'], time_format='timedelta')

In [13]:
sf = 250
epoch = meditation_data[:60*sf]

In [15]:
meditation_data.groupby(pd.Grouper(freq='10s'))

In [14]:
electrodes = epoch.columns.values
bp = bandpower.get_bandpower_epochs_for_all_electrodes(epoch, cfg_mr)

In [15]:
bp

channel                           E1            E2            E3  \
              time                                                 
gamma         00:00:00  1.272676e-08  3.139696e-08  2.574487e-08   
              00:00:10  2.424481e-08  1.543658e-08  1.296063e-08   
              00:00:20  1.553556e-07  3.494884e-08  3.069125e-08   
              00:00:30  3.794091e-08  9.634466e-09  1.444760e-08   
              00:00:40  2.795332e-08  5.058870e-09  5.315167e-09   
              00:00:50  3.618453e-08  8.983504e-09  9.265785e-09   
beta          00:00:00  4.940330e-04  2.820206e-04  1.740999e-04   
              00:00:10  5.823026e-02  7.564360e-02  7.405468e-02   
              00:00:20  3.278126e-01  1.338681e-01  1.098056e-01   
              00:00:30  1.770668e-01  4.837945e-02  5.713779e-02   
              00:00:40  1.754116e-01  3.252439e-02  2.399566e-02   
              00:00:50  2.037076e-01  5.296190e-02  3.715937e-02   
alpha         00:00:00  1.972494e-04  1.894607e-04  1.566209e-04   
              00:00:10  1.944186e-02  1.968200e-02  2.271470e-02   
              00:00:20  8.177856e-02  2.009265e-02  2.352832e-02   
              00:00:30  7.054011e-02  2.530927e-02  3.732018e-02   
              00:00:40  1.189500e-01  2.780984e-02  2.355838e-02   
              00:00:50  1.081392e-01  3.034525e-02  1.910276e-02   
theta         00:00:00  5.387646e-04  7.393132e-04  9.953309e-04   
              00:00:10  3.633958e-02  3.927442e-02  9.715220e-02   
              00:00:20  9.778711e-02  6.137911e-02  7.581234e-02   
              00:00:30  1.093403e-01  7.315580e-02  1.477338e-01   
              00:00:40  9.327299e-02  6.792284e-02  9.004646e-02   
              00:00:50  9.378097e-02  8.036894e-02  8.523009e-02   
delta         00:00:00  6.226015e-03  7.795655e-03  8.398963e-03   
              00:00:10  1.843006e-01  2.378258e-01  3.783199e-01   
              00:00:20  1.440856e-01  1.908258e-01  2.892425e-01   
              00:00:30  2.702525e-01  2.664046e-01  5.884896e-01   
              00:00:40  2.240744e-01  2.563514e-01  3.513504e-01   
              00:00:50  2.784200e-01  2.895234e-01  2.366025e-01   
...                              ...           ...           ...   
beta / theta  00:00:00  9.169737e-01  3.814630e-01  1.749165e-01   
              00:00:10  1.602392e+00  1.926027e+00  7.622543e-01   
              00:00:20  3.352309e+00  2.181004e+00  1.448387e+00   
              00:00:30  1.619411e+00  6.613207e-01  3.867619e-01   
              00:00:40  1.880626e+00  4.788431e-01  2.664809e-01   
              00:00:50  2.172163e+00  6.589847e-01  4.359889e-01   
beta / delta  00:00:00  7.934979e-02  3.617664e-02  2.072873e-02   
              00:00:10  3.159527e-01  3.180631e-01  1.957462e-01   
              00:00:20  2.275124e+00  7.015200e-01  3.796316e-01   
              00:00:30  6.551902e-01  1.816014e-01  9.709227e-02   
              00:00:40  7.828274e-01  1.268742e-01  6.829553e-02   
              00:00:50  7.316557e-01  1.829279e-01  1.570540e-01   
alpha / theta 00:00:00  3.661141e-01  2.562658e-01  1.573557e-01   
              00:00:10  5.350052e-01  5.011404e-01  2.338053e-01   
              00:00:20  8.362918e-01  3.273533e-01  3.103495e-01   
              00:00:30  6.451430e-01  3.459639e-01  2.526178e-01   
              00:00:40  1.275289e+00  4.094328e-01  2.616248e-01   
              00:00:50  1.153104e+00  3.775744e-01  2.241317e-01   
alpha / delta 00:00:00  3.168148e-02  2.430337e-02  1.864765e-02   
              00:00:10  1.054900e-01  8.275805e-02  6.004100e-02   
              00:00:20  5.675691e-01  1.052932e-01  8.134462e-02   
              00:00:30  2.610155e-01  9.500312e-02  6.341689e-02   
              00:00:40  5.308504e-01  1.084833e-01  6.705097e-02   
              00:00:50  3.884033e-01  1.048110e-01  8.073779e-02   
theta / delta 00:00:00  8.653443e-02  9.483656e-02  1.185064e-01   
              00:00:10  1.971756e-01  1.651394e-01  2.56

In [6]:
vars(data)

{'_new_trigger': array([0., 0., 0., ..., 0., 0., 0.]),
 'event_id': {},
 'preload': False,
 '_last_samps': array([731186]),
 '_first_samps': array([0]),
 'info': <Info | 7 non-empty values
  bads: []
  ch_names: E1, E2, E3, E4, E5, E6, E7, E8, E9, E10, E11, E12, E13, E14, ...
  chs: 256 EEG, 3 STIM
  custom_ref_applied: False
  highpass: 0.0 Hz
  lowpass: 125.0 Hz
  meas_date: 2015-05-12 08:43:36 UTC
  nchan: 259
  projs: []
  sfreq: 250.0 Hz
 >,
 'buffer_size_sec': 1.0,
 'verbose': None,
 '_cals': array([1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06,
        1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06,
        1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06,
        1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06,
        1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06,
        1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06,
        1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1

In [8]:
# length of recording: 48.7458 minutes
731187 / 250 / 60

48.7458

In [23]:
meditation_data.groupby(pd.Grouper(freq='10s'))

In [24]:
meditation_data

channel,E1,E2,E3,E4,E5,E6,E7,E8,E9,E10,...,E250,E251,E252,E253,E254,E255,E256,nois,sync,STI 014
time,,,,,,,,,,,,,,,,,,,,,
00:00:00,1256.471191,1525.020386,1739.435791,2906.021484,2790.609619,1950.713867,-265.795715,-978.799072,-856.525024,2005.824219,...,3519.403809,3039.684814,7.180149e+04,2878.648926,3696.833496,3301.607422,910.430481,0.0,0.0,0.0
00:00:00.004000,1256.471191,1525.020386,1739.435791,2906.021484,2790.609619,1950.713867,-265.795715,-978.799072,-856.525024,2005.824219,...,3519.403809,3039.684814,7.180149e+04,2878.648926,3696.833496,3301.607422,910.430481,0.0,0.0,0.0
00:00:00.008000,1256.471191,1525.020386,1739.435791,2906.021484,2790.609619,1950.713867,-265.795715,-978.799072,-856.525024,2005.824219,...,3519.403809,3039.684814,7.180149e+04,2878.648926,3696.833496,3301.607422,910.430481,0.0,0.0,0.0
00:00:00.012000,1256.471191,1525.020386,1739.435791,2906.021484,2790.609619,1950.713867,-265.795715,-978.799072,-856.525024,2005.824219,...,3519.403809,3039.684814,7.180149e+04,2878.648926,3696.833496,3301.607422,910.430481,0.0,0.0,0.0
00:00:00.016000,1256.471191,1525.020386,1739.435791,2906.021484,2790.609619,1950.713867,-265.795715,-978.799072,-856.525024,2005.824219,...,3519.403809,3039.684814,7.180149e+04,2878.648926,3696.833496,3301.607422,910.430481,0.0,0.0,0.0
00:00:00.020000,1256.471191,1525.020386,1739.435791,2906.021484,2790.609619,1950.713867,-265.795715,-978.799072,-856.525024,2005.824219,...,3519.403809,3039.684814,7.180149e+04,2878.648926,3696.833496,3301.607422,910.430481,0.0,0.0,0.0
00:00:00.024000,1256.471191,1525.020386,1739.435791,2906.021484,2790.609619,1950.713867,-265.795715,-978.799072,-856.525024,2005.824219,...,3519.403809,3039.684814,7.180149e+04,2878.648926,3696.833496,3301.607422,910.430481,0.0,0.0,0.0
00:00:00.028000,1256.471191,1525.020386,1739.435791,2906.021484,2790.609619,1950.713867,-265.795715,-978.799072,-856.525024,2005.824219,...,3519.403809,3039.684814,7.180149e+04,2878.648926,3696.833496,3301.607422,910.430481,0.0,0.0,0.0
00:00:00.032000,1256.471191,1525.020386,1739.435791,2906.021484,2790.609619,1950.713867,-265.795715,-978.799072,-856.525024,2005.824219,...,3519.403809,3039.684814,7.180149e+04,2878.648926,3696.833496,3301.607422,910.430481,0.0,0.0,0.0
